## Combining data files

Load RMagic

In [2]:
%load_ext rpy2.ipython

Load in metadata and process.

In [3]:
%%R
metadata <- read.table("SRX190717_metadata.txt",header=TRUE,row.names=NULL,sep=",",stringsAsFactors=FALSE)
names(metadata) <- toupper(names(metadata))
metadata$MID <- paste("MID",metadata$MID,sep="")
head(metadata)

        RUN SUBJECT VISIT CELLTYPE   MID
1 SRR747232      11    V3     PBMC MID33
2 SRR747232      25    V1     PBMC MID34
3 SRR747232      25    V3     PBMC MID35
4 SRR747232      29    V1     PBMC MID36
5 SRR747232      29    V3     PBMC MID37
6 SRR747232      11    V1     PBMC MID38


Extract runs.

In [4]:
%%R
runs <- sort(unique(metadata$RUN))
nruns <- length(runs)

Check the processed ChangeO databases.

In [5]:
%%R
list.files(path=".",pattern="_db-pass.tab")

 [1] "SRR747232_db-pass.tab" "SRR747758_db-pass.tab" "SRR747760_db-pass.tab"
 [4] "SRR747761_db-pass.tab" "SRR747763_db-pass.tab" "SRR747766_db-pass.tab"
 [7] "SRR747767_db-pass.tab" "SRR747768_db-pass.tab" "SRR747785_db-pass.tab"
[10] "SRR765688_db-pass.tab" "SRR770500_db-pass.tab"


Merge:
- Read in the database for each run
- Merge with the metadata
- Save to a file (append if the second run or higher)

In [6]:
%%R
for(i in 1:nruns){
    df <- read.table(paste(runs[i],"_db-pass.tab",sep=""),header=TRUE,row.names=NULL,stringsAsFactors=FALSE,sep="\t")
    df$RUN <- runs[i]
    df <- merge(df,metadata,all.x=TRUE,all.y=FALSE)
    if(i==1){
        h <- names(df)
        write.table(df,file="SRX190717_db-pass.tab",col.names=TRUE,row.names=FALSE,sep="\t",quote=F)
    }else{
        if(names(df)!=h){
            stop("Names of dataframes don't match")
        }
        write.table(df,file="SRX190717_db-pass.tab",col.names=FALSE,row.names=FALSE,sep="\t",quote=F,append=TRUE)
    }    
}